In [1]:
import json
import collections
import requests
import urllib.parse
import re

import threading
import functools
def threadify(func):
    "function runs in a new thread."

    @functools.wraps(func)
    def run(*args, **kwds):
        new_thread = threading.Thread(
            target = func,
            args   = args,
            kwargs = kwds)
        new_thread.start()
        return new_thread

    return run



In [2]:
with open("name-visit-count-all.json") as f:
    data = json.load(f)

In [3]:
data = collections.Counter({k:v for k,v in data.items() if v is not None})

In [4]:
popular = data.most_common(100000)

In [5]:
def load(link):
    title = (urllib.parse.unquote(link)
        .replace("/wiki/", "")
        .replace("_", " "))
    endpoint = "https://en.wikipedia.org/w/api.php"
    params   = {
        "action"      : "query",
        "prop"        : "categories|extracts",
        "titles"      : title,
        "explaintext" : "true",
        "format"      : "json",
        "cllimit"     : "5000"
    }
    resp = requests.get(endpoint, params=params)
    pages   = resp.json()["query"]["pages"]
    extract = pages[next(iter(pages))]["extract"]
    # header  = re.split("==* .* ==*", extract)[0]
    return extract

def loader(link, headers):
    counter = 0
    for i in range(20):
        try:
            headers[link] = load(link)
            break
        except:
            pass
    else:
        print("FAILED", link)
                
@threadify
def threaded_loader(links, headers):
    while links:
        link = links.pop()
        loader(link, headers)


In [6]:
links = list(next(zip(*popular)))
headers = {}
threads = []
for i in range(20):
    t = threaded_loader(links, headers)
    threads.append(t)

In [24]:
with open("headers-100000.json", "w") as f:
    json.dump(headers, f)